In [22]:
import time
import numpy as np

In [23]:
TEST_COUNT = 5

# Count test

In [24]:
def count_test(test_count, data):
    count_time = []
    group_count_time = []

    for i in range(test_count):
        t0 = time.time()
        data.count()
        t1 = time.time()
        count_time.append(t1 - t0)

    print("Count Time")
    print(count_time)
    print("Average Count Time : %f" %(np.mean(count_time)))


## Test 1

196M 텍스트 파일 하나를 읽어서 count 테스트  

count의 결과에서  

* cache를 할 경우 파일을 처음 읽을 때 더 느림
* 두번째 이후의 처리속도는 cache여부와 무관하게 비슷했다. cache가 동작하지 않았거나 cache를 하지 않았지만 알아서 메모리에 적재했거나.


In [25]:
test_file = "hdfs://ssgslab01/user/ahthwa/pyspark_test/round2_purchaseRecord.tsv"

In [26]:
purchase_record = sc.textFile(test_file)
purchase_record_cache = sc.textFile(test_file)
purchase_record_cache.cache()
purchase_record_cache_2 = sc.textFile(test_file).cache()


In [27]:
print("---- Not Cached ----")
count_test(TEST_COUNT, purchase_record)
print("\n---- Cached-1 ----")
count_test(TEST_COUNT, purchase_record_cache)
print("\n---- Cached-2 ----")
count_test(TEST_COUNT, purchase_record_cache_2)


---- Not Cached ----
Count Time
[8.588986873626709, 8.985455989837646, 9.038344860076904, 8.993307113647461, 9.006339073181152]
Average Count Time : 8.922487

---- Cached-1 ----
Count Time
[11.52690315246582, 5.477094888687134, 5.453290939331055, 5.580351829528809, 5.484626054763794]
Average Count Time : 6.704453

---- Cached-2 ----
Count Time
[11.220871925354004, 5.47505521774292, 5.512097120285034, 5.472071170806885, 5.509972095489502]
Average Count Time : 6.638014


## Test 2

전체 용량 369.8M인 로그 파일 여러개를 읽어서 count, groupby하는 테스트

두 번째 이후 count에서 cache를 통해 성능 향상 가능하다.

In [28]:
test_file = "hdfs://ssgslab01/data/wifimon/dashboard/base_log/20160701/part-*"

In [29]:
baselog = sc.textFile(test_file)
baselog_cache = sc.textFile(test_file)
baselog_cache.cache()
baselog_cache_2 = sc.textFile(test_file).cache()

In [30]:
print("---- Not Cached ----")
count_test(TEST_COUNT, baselog)
print("\n---- Cached-1 ----")
count_test(TEST_COUNT, baselog_cache)
print("\n---- Cached-2 ----")
count_test(TEST_COUNT, baselog_cache_2)

---- Not Cached ----
Count Time
[9.249214172363281, 9.96982192993164, 9.994314193725586, 10.000025033950806, 10.008059978485107]
Average Count Time : 9.844287

---- Cached-1 ----
Count Time
[12.327146053314209, 3.0349879264831543, 2.962813138961792, 2.9769110679626465, 2.971198081970215]
Average Count Time : 4.854611

---- Cached-2 ----
Count Time
[13.09050989151001, 3.0133249759674072, 2.985042095184326, 2.971369981765747, 2.985313892364502]
Average Count Time : 5.009112


# Group by Test

In [31]:
def groupby_count(data, key_idx):
    return data.map(lambda x:x.strip().split('\t')).map(lambda x:(x[key_idx], 1)).reduceByKey(lambda a,b:a+b)
    

## Test 3

group by할 데이터를 한번 읽어서 재사용, 한번 읽어서 cache, 매번 읽는 경우를 비교함.

* 데이터를 로딩하는 것이 매번 읽는 것 보다 빠르다(당연히)  
* cache를 명시적으로 사용하지 않아도 cache 한 것과 비슷한 속도를 낸다.

In [32]:
test_file = "hdfs://ssgslab01/data/wifimon/dashboard/base_log/20160708/part-*"
baselog_2 = sc.textFile(test_file)

def groupby_test_1(test_count, data):
    group_count_time = []

    for i in range(test_count):
        t0 = time.time()
        groupby_count(data, 0)
        t1 = time.time()
        group_count_time.append(t1 - t0)
    
    print("Group by Time")
    print(group_count_time)
    print("Average Group By Time : %f" %(np.mean(group_count_time)))

In [33]:
groupby_test_1(TEST_COUNT, baselog_2)

Group by Time
[0.06061100959777832, 0.014039039611816406, 0.014146089553833008, 0.013869047164916992, 0.013686895370483398]
Average Group By Time : 0.023270


In [34]:
test_file = "hdfs://ssgslab01/data/wifimon/dashboard/base_log/20160715/part-*"
baselog_3 = sc.textFile(test_file).cache()

def groupby_test_2(test_count, data):
    group_count_time = []

    for i in range(test_count):
        t0 = time.time()
        groupby_count(data, 0)
        t1 = time.time()
        group_count_time.append(t1 - t0)
    
    print("Group by Time")
    print(group_count_time)
    print("Average Group By Time : %f" %(np.mean(group_count_time)))

In [35]:
groupby_test_2(TEST_COUNT, baselog_3)

Group by Time
[0.04703497886657715, 0.025379180908203125, 0.013731002807617188, 0.013928890228271484, 0.013622045516967773]
Average Group By Time : 0.022739


In [36]:
test_file = "hdfs://ssgslab01/data/wifimon/dashboard/base_log/20160722/part-*"
def groupby_test_3(test_count, test_file):
    group_count_time = []

    for i in range(test_count):
        data = sc.textFile(test_file)
        t0 = time.time()
        groupby_count(data, 0)
        t1 = time.time()
        group_count_time.append(t1 - t0)
    
    print("Group by Time")
    print(group_count_time)
    print("Average Group By Time : %f" %(np.mean(group_count_time)))

In [37]:
groupby_test_3(TEST_COUNT, test_file)

Group by Time
[0.04830193519592285, 0.04689216613769531, 0.04421401023864746, 0.052915096282958984, 0.04367995262145996]
Average Group By Time : 0.047201
